In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from collections import Counter
from imblearn.over_sampling import SMOTE

### Load the data

In [3]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Oversampling by ethnicity

In [4]:
e = data["Ethnicity"]
print('Original dataset shape %s' % Counter(e))

ethnicity_counts = dict(data["Ethnicity"].value_counts())
num_ethnicities = len(ethnicity_counts)
max_count = max(ethnicity_counts.values())

strategy_over = {ethnicity: 10*max_count for ethnicity in range(num_ethnicities)}
over = SMOTE(sampling_strategy=strategy_over)
print(strategy_over, sep='\n')

data_over, e_over=over.fit_resample(data, e)
print('Oversampled dataset shape %s' % Counter(e_over))

Original dataset shape Counter({0: 1278, 1: 454, 3: 211, 2: 206})
{0: 12780, 1: 12780, 2: 12780, 3: 12780}
Oversampled dataset shape Counter({0: 12780, 3: 12780, 1: 12780, 2: 12780})


### Split the data

In [5]:
X, y = data_over.drop("Diagnosis", axis=1), data_over["Diagnosis"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_train.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MMSE,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness
32101,6037,82,1,2,0,33.259295,0,17.957938,8.341254,5.676656,...,23.516844,3.703507,0,0,3.639448,0,0,0,0,0
46502,4839,86,0,3,1,30.600376,0,14.149346,0.845528,9.764606,...,3.316463,8.140269,0,0,9.183472,0,0,0,0,1
30948,5748,60,0,2,1,23.262465,0,16.120964,8.619289,4.983863,...,26.942157,0.156727,0,0,5.094553,0,0,0,0,0
28710,5136,68,0,2,1,29.481026,0,4.865708,1.645098,5.627643,...,11.530320,6.993090,0,0,5.767891,0,0,0,0,0
46552,6600,80,0,3,0,37.544310,0,13.408678,4.246339,7.073541,...,13.689179,6.583770,0,0,4.490818,0,0,0,0,0


### Train the model

In [6]:
classifier = MLPClassifier(max_iter=5000, random_state=12)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [7]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93     10954
           1       0.73      0.07      0.13      1826

    accuracy                           0.86     12780
   macro avg       0.80      0.53      0.53     12780
weighted avg       0.85      0.86      0.81     12780



In [8]:
X["Ethnicity"].value_counts()

Ethnicity
0    12780
3    12780
1    12780
2    12780
Name: count, dtype: int64

### Compare performance based on ethnicity

In [9]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [10]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.85      0.99      0.92      2724
           1       0.69      0.08      0.14       499

    accuracy                           0.85      3223
   macro avg       0.77      0.53      0.53      3223
weighted avg       0.83      0.85      0.80      3223



In [11]:
# Ethncity 1
eth1_results = results[results["Ethnicity"] == 1]
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      2809
           1       0.65      0.09      0.16       380

    accuracy                           0.89      3189
   macro avg       0.77      0.54      0.55      3189
weighted avg       0.86      0.89      0.85      3189



In [12]:
# Ethnicity 2  
eth2_results = results[results["Ethnicity"] == 2]
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      2578
           1       0.81      0.07      0.14       615

    accuracy                           0.82      3193
   macro avg       0.81      0.54      0.52      3193
weighted avg       0.82      0.82      0.75      3193



In [13]:
# Ethncity 3
eth3_results = results[results["Ethnicity"] == 3]
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2843
           1       0.85      0.03      0.06       332

    accuracy                           0.90      3175
   macro avg       0.87      0.52      0.50      3175
weighted avg       0.89      0.90      0.85      3175



In [14]:
from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 1)
eo = equal_opportunity(results, 0, 1)
di = disparate_impact(results, 0, 1)

print(dp, eo, di)

from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 2)
eo = equal_opportunity(results, 0, 2)
di = disparate_impact(results, 0, 2)

print(dp, eo, di)


from fairness_metrics import demographic_parity, equal_opportunity, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

dp = demographic_parity(results, 0, 3)
eo = equal_opportunity(results, 0, 3)
di = disparate_impact(results, 0, 3)

print(dp, eo, di)

0.00013163851421856693 0.04276094276094278 1.0077739855896855
0.0007867038498250746 0.11610845295055816 0.955930782587869
0.012970358227405875 0.15524475524475523 4.167760567077973
